## Úvod

Tato lekce se zaměří na:
- Co je volání funkcí a k čemu se používá
- Jak vytvořit volání funkce pomocí Azure OpenAI
- Jak integrovat volání funkce do aplikace

## Výukové cíle

Po dokončení této lekce budete vědět a rozumět:

- Účelu používání volání funkcí
- Nastavení volání funkce pomocí služby Azure Open AI
- Navrhování efektivních volání funkcí pro konkrétní použití ve vaší aplikaci


## Pochopení volání funkcí

V této lekci chceme vytvořit funkci pro náš vzdělávací startup, která umožní uživatelům pomocí chatbota najít technické kurzy. Budeme doporučovat kurzy, které odpovídají jejich úrovni znalostí, aktuální pracovní pozici a technologii, o kterou mají zájem.

K dokončení tohoto úkolu použijeme kombinaci:
 - `Azure Open AI` pro vytvoření chatovacího prostředí pro uživatele
 - `Microsoft Learn Catalog API` pro pomoc uživatelům najít kurzy na základě jejich požadavků
 - `Function Calling` pro zpracování dotazu uživatele a jeho odeslání do funkce, která provede API požadavek

Než začneme, podívejme se, proč bychom vlastně chtěli použít volání funkcí:

print("Zprávy v dalším požadavku:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # získání nové odpovědi od GPT, která už vidí odpověď funkce


print(second_response.choices[0].message)


### Proč používat Function Calling

Pokud jste absolvovali jakoukoli jinou lekci v tomto kurzu, pravděpodobně už chápete sílu využití velkých jazykových modelů (LLM). Doufáme, že si také uvědomujete některá jejich omezení.

Function Calling je funkce služby Azure Open AI, která pomáhá překonat následující omezení:
1) Konzistentní formát odpovědí
2) Možnost využívat data z jiných zdrojů aplikace v rámci konverzace

Před zavedením function calling byly odpovědi z LLM nestrukturované a nekonzistentní. Vývojáři museli psát složitý validační kód, aby dokázali zpracovat každou variantu odpovědi.

Uživatelé nemohli získat odpovědi typu „Jaké je aktuální počasí ve Stockholmu?“. Důvodem bylo, že modely měly omezené znalosti pouze na období, kdy byla data trénována.

Podívejme se na následující příklad, který tento problém ilustruje:

Představme si, že chceme vytvořit databázi studentských údajů, abychom jim mohli doporučit vhodný kurz. Níže máme dva popisy studentů, které obsahují velmi podobná data.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Toto chceme poslat LLM, aby zpracoval data. Později to můžeme použít v naší aplikaci k odeslání na API nebo uložení do databáze.

Vytvořme dva totožné promptové dotazy, kterými LLM sdělíme, o jaké informace máme zájem:


Chceme to poslat LLM, aby analyzoval části, které jsou důležité pro náš produkt. Takže můžeme vytvořit dva identické promptu, abychom LLM instruovali:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po vytvoření těchto dvou promptů je odešleme do LLM pomocí `openai.ChatCompletion`. Prompt uložíme do proměnné `messages` a přiřadíme roli `user`. To slouží k napodobení zprávy od uživatele, která je zaslána chatbotu.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

I když jsou zadání stejná a popisy podobné, můžeme dostat různé formáty vlastnosti `Grades`.

Pokud spustíte výše uvedenou buňku několikrát, formát může být `3.7` nebo `3.7 GPA`.

Je to proto, že LLM pracuje s nestrukturovanými daty ve formě napsaného zadání a vrací také nestrukturovaná data. Potřebujeme mít strukturovaný formát, abychom věděli, co očekávat při ukládání nebo používání těchto dat.

Použitím funkčního volání si můžeme zajistit, že dostaneme zpět strukturovaná data. Při použití funkčního volání LLM ve skutečnosti žádné funkce nevolá ani nespouští. Místo toho vytvoříme strukturu, kterou má LLM dodržovat ve svých odpovědích. Tyto strukturované odpovědi pak využíváme k tomu, abychom věděli, jakou funkci v našich aplikacích spustit.


![Diagram toku volání funkcí](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.cs.png)


### Příklady použití volání funkcí

**Volání externích nástrojů**  
Chatboti jsou skvělí v poskytování odpovědí na dotazy uživatelů. Díky volání funkcí mohou chatboti využít zprávy od uživatelů k provedení určitých úkolů. Například student může požádat chatbota: „Pošli e-mail mému vyučujícímu, že potřebuji s tímto předmětem více pomoci.“ To může vyvolat volání funkce `send_email(to: string, body: string)`

**Vytváření API nebo databázových dotazů**  
Uživatelé mohou najít informace pomocí přirozeného jazyka, který se převede na formátovaný dotaz nebo API požadavek. Příkladem může být učitel, který se zeptá: „Kteří studenti dokončili poslední úkol?“, což může vyvolat funkci s názvem `get_completed(student_name: string, assignment: int, current_status: string)`

**Vytváření strukturovaných dat**  
Uživatelé mohou vzít blok textu nebo CSV a použít LLM k extrakci důležitých informací. Například student může převést článek z Wikipedie o mírových dohodách a vytvořit z něj AI kartičky. To lze provést pomocí funkce `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Vytvoření vašeho prvního volání funkce

Proces vytvoření volání funkce zahrnuje 3 hlavní kroky:
1. Zavolání Chat Completions API se seznamem vašich funkcí a uživatelskou zprávou
2. Přečtení odpovědi modelu za účelem provedení akce, například spuštění funkce nebo API volání
3. Opětovné zavolání Chat Completions API s odpovědí z vaší funkce, abyste tuto informaci použili k vytvoření odpovědi pro uživatele.


![Tok volání funkce](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.cs.png)


### Prvky volání funkce

#### Vstup uživatele

Prvním krokem je vytvořit zprávu od uživatele. To lze dynamicky nastavit podle hodnoty textového vstupu, nebo můžete hodnotu přiřadit přímo zde. Pokud s Chat Completions API pracujete poprvé, je potřeba definovat `role` a `content` zprávy.

`role` může být buď `system` (nastavení pravidel), `assistant` (model) nebo `user` (koncový uživatel). Pro volání funkce nastavíme tuto hodnotu na `user` a přidáme ukázkovou otázku.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Vytváření funkcí.

Nyní si definujeme funkci a její parametry. V tomto příkladu použijeme pouze jednu funkci s názvem `search_courses`, ale můžete si vytvořit i více funkcí.

**Důležité** : Funkce jsou zahrnuty ve zprávě systému pro LLM a budou započítány do počtu dostupných tokenů, které máte k dispozici.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definice**

`name` - Název funkce, kterou chceme zavolat.

`description` - Popis toho, jak funkce funguje. Je důležité být konkrétní a jasný.

`parameters` - Seznam hodnot a formát, který chcete, aby model použil ve své odpovědi.

`type` - Datový typ, ve kterém budou vlastnosti uloženy.

`properties` - Seznam konkrétních hodnot, které model použije ve své odpovědi.

`name` - Název vlastnosti, kterou model použije ve své formátované odpovědi.

`type` - Datový typ této vlastnosti.

`description` - Popis konkrétní vlastnosti.

**Volitelné**

`required` - Povinná vlastnost, aby bylo možné volání funkce dokončit.


### Volání funkce
Po definování funkce ji nyní musíme zahrnout do volání Chat Completion API. Uděláme to tak, že do požadavku přidáme `functions`. V tomto případě `functions=functions`.

Je zde také možnost nastavit `function_call` na `auto`. To znamená, že necháme LLM rozhodnout, kterou funkci má zavolat na základě zprávy od uživatele, místo abychom ji určovali sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nyní se podívejme na odpověď a zjistíme, jak je naformátovaná:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Můžete vidět, že je volána funkce s názvem a z uživatelské zprávy LLM dokázal najít data, která odpovídají argumentům funkce.


## 3. Integrace volání funkcí do aplikace.

Poté, co jsme otestovali naformátovanou odpověď z LLM, můžeme ji nyní integrovat do aplikace.

### Řízení toku

Abychom to integrovali do naší aplikace, postupujme následovně:

Nejprve zavoláme služby Open AI a uložíme zprávu do proměnné s názvem `response_message`.


In [ ]:
response_message = response.choices[0].message

Nyní definujeme funkci, která zavolá Microsoft Learn API a získá seznam kurzů:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Jako osvědčený postup nejprve zjistíme, zda model chce zavolat nějakou funkci. Poté vytvoříme jednu z dostupných funkcí a přiřadíme ji k funkci, kterou model požaduje.

Následně vezmeme argumenty funkce a namapujeme je na argumenty z LLM.

Nakonec připojíme zprávu o volání funkce a hodnoty, které byly vráceny zprávou `search_courses`. Tím získá LLM všechny potřebné informace k tomu, aby mohl uživateli odpovědět přirozeným jazykem.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kódová výzva

Skvělá práce! Pokud chcete dále rozvíjet své znalosti o Azure Open AI Function Calling, můžete si vyzkoušet: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Přidejte více parametrů do funkce, které mohou studentům pomoci najít více kurzů. Dostupné parametry API najdete zde: 
 - Vytvořte další volání funkce, které bude brát v úvahu více informací od studenta, například jeho rodný jazyk 
 - Přidejte zpracování chyb pro případ, že volání funkce a/nebo API nevrátí žádné vhodné kurzy



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro kritické informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
